In [1]:
import time
import pycurl
import urllib
import json
import requests
import numpy as np

In [2]:
API_ENDPOINT_URL = 'http://lauzhack.sqpub.ch/'
API_KEY = '98423jhHJDmM'

class SwissQuoteStream:
    def __init__(self):
        
        self.conn = None
        self.buffer = ''
        
        
    def setup_connection(self):
        if self.conn:
            self.conn.close()
            self.buffer = ''
        self.conn = pycurl.Curl()
        self.conn.setopt(pycurl.URL, API_ENDPOINT_URL + 'prices')
        self.conn.setopt(pycurl.WRITEFUNCTION, self.callback)
    
    def start(self, callback):
        self.callback = callback
        self.setup_connection()
        backoff_network_error = 0.25
        backoff_http_error = 5
        backoff_rate_limit = 60
        while True:
            self.setup_connection()
            try:
                self.conn.perform()
            except:
                print('Network error')
                time.sleep(backoff_network_error)
                backoff_network_error = min(backoff_network_error + 1, 16)
                continue
            # HTTP Error
            sc = self.conn.getinfo(pycurl.HTTP_CODE)

            # HTTP error, use exponential back off up to 320 seconds
           # print(HTTP error %s, %s' % (sc, self.conn.errstr()))
           # print 'Waiting %s seconds' % backoff_http_error
            time.sleep(backoff_http_error)
            backoff_http_error = min(backoff_http_error * 2, 320)
            
    def buy(self, btc_amount):
        #data = str.encode('BUY 0.00000001 BTC ' + API_KEY)
        data = str.encode('BUY ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data=data)
        return str(r.status_code) + ' ' + str(r.reason)

        
    def sell(self, btc_amount):
        #data = str.encode('SELL 0.00000001 BTC ' + API_KEY)
        data = str.encode('SELL ' + str(btc_amount) +' BTC ' + API_KEY)
        r = requests.post(API_ENDPOINT_URL, data=data)
        return str(r.status_code) + ' ' + str(r.reason)
        


In [3]:
stream = SwissQuoteStream()

In [4]:
def extract_price(data):
    return float(data.decode("utf-8").split(' ')[1][:-1])

WINDOW_MAX_SIZE = 20

window = []
epsilon = 0.01
ignore_count = 0
ignore_n = 1

normalizer_max = 5

def diff_normalizer(diff):
    if(diff > normalizer_max):
        diff_tmp = normalizer_max
    elif(diff < -normalizer_max):
        diff_tmp = -normalizer_max
    else:
        diff_tmp = diff
        
    normalized = (diff_tmp + normalizer_max) / (normalizer_max * 2)
    
    return normalized

def handle_new_price(data):
    global ignore_count

    new_price = extract_price(data)
    window.append(new_price)
    window_resized = window[-WINDOW_MAX_SIZE:]
    window_size = len(window_resized)
    
    weights = np.exp(np.linspace(-1., 0., window_size))
    weights /= weights.sum()
    
    a = np.convolve(window_resized, weights, mode='full')[:len(window_resized)]
    ema = a[window_size -1]
    
    diff = new_price - ema
    # diff > 0 -> price is above ema -> SELL SELL SELL!

    ignore_count = (ignore_count + 1 ) % ignore_n
    
    if(ignore_count == 0):
        big_enough = abs(diff) > epsilon
        
        if(not big_enough):
            print('Just HODL. Diff is: ' + str(diff))
            return
        
        END = 'http://lauzhack.sqpub.ch/teams/'

        r = requests.get(END)
        assets = r.json()[12]["assets"]
        cash = r.json()[12]["cash"]
        total = assets + cash
        
        current_percent_btc = assets / total #percentage of current bitcoin compared to total balance
        normalized_diff = 1 - diff_normalizer(diff) #percentage of bitcoin we want to have, compared to total balance
        percent_btc_diff = normalized_diff - current_percent_btc #percentage of cash to add
        amount_of_btc = percent_btc_diff * total # amount of bitcoin we want to add
        amount_in_btc = amount_of_btc / new_price #amount of equivalent bitcoin we want to add in cash
        
        #print(str(total) + ' ' + str(assets) + ' ' + str(cash) + ' ||| ' + str(diff) + ' ' + str(current_percent_btc) + ' ' + str(percent_btc_diff) + ' ' + str(amount_of_btc) + ' ' + str(amount_in_btc))
        
        if(amount_in_btc < 0):
            aa = stream.sell(abs(amount_in_btc))
            print('Diff: ' + str(normalized_diff) + '. SELL ' + str(abs(amount_in_btc)))
        else:
            aa = stream.buy(abs(amount_in_btc))
            print('Diff: ' + str(normalized_diff) + '. BUY ' + str(abs(amount_in_btc)))


In [ ]:
stream.start(handle_new_price)

Just HODL. Diff is: 0.0
Diff: 0.49415153137097145. BUY 4.60167050392293
Diff: 0.5358182944335113. BUY 1.1405796085710191
Diff: 0.5292975662761364. SELL 0.17849680062969478
Diff: 0.4908756537281078. SELL 1.0520912535284321
Diff: 0.4892880442035221. SELL 0.04338172105951778
Diff: 0.4889757630281565. SELL 0.008524070054266598
Diff: 0.49061435861485736. BUY 0.04483748402383399
Diff: 0.5782710215489715. BUY 2.4006091095114677
Diff: 0.4462128702967221. SELL 3.616000578530927
Diff: 0.5560420549621995. BUY 3.0079709738881615
Diff: 0.4627078692373743. SELL 2.5558968307829053
Diff: 0.5321317887356599. BUY 1.9026035058706823
Diff: 0.6044960405527491. BUY 1.9782068233185073
Diff: 0.46365072780758965. SELL 3.854959692251692
Diff: 0.4647742444258256. BUY 0.030922920374246217
Diff: 0.5479640726508024. BUY 2.2782922833639185
Diff: 0.5462655150507999. SELL 0.04531020800035615
Diff: 0.6009025071506585. BUY 1.4953942098960393
Diff: 0.5986923509448389. SELL 0.06332394797874098
Diff: 0.46433641275439186. S

Diff: 0.41701032302189556. BUY 0.12422359344106935
Diff: 0.32267028541818943. SELL 2.5812149905806083
Diff: 0.32969016045794886. BUY 0.19201738360581375
Diff: 0.3370893982677444. BUY 0.20229997978175634
Diff: 0.34488850008292504. BUY 0.21323221770837936
Diff: 0.35310907505136124. BUY 0.22475521013481553
Diff: 0.36185691507171214. BUY 0.23917063040426892
Diff: 0.37099448427097736. BUY 0.2498254516322446
Diff: 0.3806258590640027. BUY 0.2633259427233313
Diff: 0.39077772531873056. BUY 0.2775558829455912
Diff: 0.40147821104046666. BUY 0.28993536240152334
Diff: 0.34964025631684303. SELL 1.4174038223236898
Diff: 0.28097636134953063. SELL 1.876543386041356
Diff: 0.2981952897972405. BUY 0.4707290365797952
Diff: 0.3173987917770319. BUY 0.524910762402439
Diff: 0.33553199238067466. BUY 0.49564787615307004
Diff: 0.34739767833762014. BUY 0.3243279900226852
Diff: 0.3599046007078869. BUY 0.34436627158665206
Diff: 0.3740584389807736. BUY 0.3869647729967013
Diff: 0.5335770685343959. BUY 4.36168979649758

Diff: 0.6127279259894294. BUY 0.37962285376563326
Diff: 0.617987245293125. BUY 0.14413697814621396
Diff: 0.6052221621071112. SELL 0.3496125098161898
Diff: 0.6487508817431717. BUY 1.1926417890692786
Diff: 0.6336535420782183. SELL 0.41342610577301386
Diff: 0.6187113496947404. SELL 0.40905850245519404
Diff: 0.6090217551726255. SELL 0.2652550918899423
Diff: 0.5997795491991382. SELL 0.252998388952368
Diff: 0.5894240698302837. SELL 0.2835142353792458
Diff: 0.5737519611236621. SELL 0.4290533950603707
